In [ ]:
pip install langchain langchain_community langchain_chroma

In [ ]:
pip install -qU langchain-openai

In [ ]:
pip install faiss-cpu

In [ ]:
import getpass
import os

HUGGINGFACE_API_KEY = getpass.getpass()

In [ ]:
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY

In [ ]:
OPENAI_API_KEY = getpass.getpass()

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
import json
from pathlib import Path
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain.docstore.document import Document

In [ ]:
file_path = './data.json'
data = json.loads(Path(file_path).read_text())

texts = []
for json in data:
    desciption = json['description']
    poc_code = json['poc_code']
    vulnerable_code = json['vulnerable_code']
    text_to_embed = f"{desciption} {poc_code} {vulnerable_code}" # need to design prompt template
    texts.append(text_to_embed)

In [ ]:
documents = [Document(page_content=text) for text in texts]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(documents)

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [ ]:
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-4.0-turbo", temperature=0)